<a href="https://colab.research.google.com/github/now1256/Seminar/blob/main/logistic_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
torch.manual_seed(1)

In [ ]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]

In [ ]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [ ]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [ ]:
# y에 맞춰서 나오는 가중치값 2x1
W = torch.zeros((2,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [ ]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [ ]:
print(hypothesis)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)


In [ ]:
losses = -(y_train * torch.log(hypothesis) +
           (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


In [ ]:
# 전체 오차에 대한 평균
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [ ]:
# cost를 찾는 함수 위에 코드와 같다
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

학습을 추가한 코드

In [ ]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [ ]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

In [ ]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [ ]:
 # 학습이 되었음
print(hypothesis)

tensor([[2.7711e-04],
        [3.1636e-02],
        [3.9014e-02],
        [9.5618e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)


In [ ]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction)

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])


로지스틱 회귀 클래스로 구현

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
torch.manual_seed(1)

In [ ]:
 # Linear에 input 넣었을때
x_data = [[1, 2, 5], [2, 3, 7], [3, 1, 4], [4, 3, 6], [5, 3, 5], [6, 2, 5]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
print(x_train.shape)


torch.Size([6, 3])


In [ ]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [ ]:
model = BinaryClassifier()

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
     # prediction = [false, true , false , true , true , true]
      prediction = hypothesis >= torch.FloatTensor([0.5])
     # prediction.float() = [0,1,0,1,1,1] y_train = [0,0,0,1,1,1]
      correct_prediction = prediction.float() == y_train
     # correct_prediction = [true , false , true, true, true, true]
      accuracy = correct_prediction.sum().item() / len(correct_prediction)
      print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/1000 Cost: 0.554989 Accuracy 83.33%
Epoch   10/1000 Cost: 0.320308 Accuracy 66.67%
Epoch   20/1000 Cost: 0.269405 Accuracy 83.33%
Epoch   30/1000 Cost: 0.227233 Accuracy 83.33%
Epoch   40/1000 Cost: 0.153119 Accuracy 83.33%
Epoch   50/1000 Cost: 0.053578 Accuracy 100.00%
Epoch   60/1000 Cost: 0.043549 Accuracy 100.00%
Epoch   70/1000 Cost: 0.038017 Accuracy 100.00%
Epoch   80/1000 Cost: 0.033866 Accuracy 100.00%
Epoch   90/1000 Cost: 0.030652 Accuracy 100.00%
Epoch  100/1000 Cost: 0.028092 Accuracy 100.00%
Epoch  110/1000 Cost: 0.026001 Accuracy 100.00%
Epoch  120/1000 Cost: 0.024255 Accuracy 100.00%
Epoch  130/1000 Cost: 0.022769 Accuracy 100.00%
Epoch  140/1000 Cost: 0.021482 Accuracy 100.00%
Epoch  150/1000 Cost: 0.020353 Accuracy 100.00%
Epoch  160/1000 Cost: 0.019351 Accuracy 100.00%
Epoch  170/1000 Cost: 0.018452 Accuracy 100.00%
Epoch  180/1000 Cost: 0.017639 Accuracy 100.00%
Epoch  190/1000 Cost: 0.016900 Accuracy 100.00%
Epoch  200/1000 Cost: 0.016223 Accuracy 100.0

In [ ]:
print(hypothesis)

tensor([[5.3996e-04],
        [2.6572e-03],
        [1.0363e-02],
        [9.9077e-01],
        [1.0000e+00],
        [9.9952e-01]], grad_fn=<SigmoidBackward0>)


In [ ]:
prediction = hypothesis >= torch.FloatTensor([0.5])
for i in range(len(prediction)):
    if  prediction[i].item() == False:
        score = '불합격'
    else :
        score = '합격'

    print('합격번호 {}번은 {} 입니다'.format(i+1 , score))

합격번호 1번은 불합격 입니다
합격번호 2번은 불합격 입니다
합격번호 3번은 불합격 입니다
합격번호 4번은 합격 입니다
합격번호 5번은 합격 입니다
합격번호 6번은 합격 입니다
